In [330]:
import numpy as np
from typing import Dict, Tuple, List
from copy import copy

edges = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,2)]
capacities = np.array([15, 8, 20, 4, 10, 15, 4, 20, 5])
costs = np.array([4, 4, 2, 2, 6, 1, 3, 2, 3])
supplies = [20, 0, 0, -5, -15]
nodes = [0, 1, 2, 3, 4]

# Define an array of supplies at each node.
class Network:
    def __init__(
        self,
        nodes: List[object],
        edges: List[Tuple[object, object]],
        capacities: List[int],
        costs: List[int],
        supplies: List[int] 
    ) -> None:
        
        M = 9999
        
        self.E = [
            *edges,
            *[(i, -1) for i in nodes],
            *[(-2, i) for i in nodes],
            (-1, -2)
        ]
        self.V = [*nodes, -1, -2]
        self.u = {
            **dict(zip(edges, capacities)),
            **{(i, -1): M for i in nodes},
            **{(-2, i): M for i in nodes},
            (-1, -2): M
        }
            
        self.c = {
            **dict(zip(edges, costs)),
            **{(i, -1): 0 for i in nodes},
            **{(-2, i): 0 for i in nodes},
            (-1, -2): M
        }
        
        self.b = {**dict(zip(nodes, supplies)), -1: 0, -2: 0}
        
N = Network(nodes, edges, capacities, costs, supplies)     

In [331]:
# Adapted from https://gist.github.com/kachayev/5990802

from collections import defaultdict
from heapq import *

def dijkstra(s, t, adj):
    def _format_path(lst):
        return [(lst[i], lst[i+1]) for i in range(len(lst) - 1)]
            
            
    g = defaultdict(list)
    for (i, j), c in adj.items():
        g[i].append((j, c))

    q, seen, distances = [(0, s, [])], set(), {s: 0}
    while q:
        cost, v1, path = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path = [*path, v1]
            if v1 == t: 
                out_path = _format_path(path)

            for v2, c in g.get(v1, ()):
                if v2 in seen: 
                    continue
                prev = distances.get(v2, None)
                nxt = cost + c
                if prev is None or nxt < prev:
                    distances[v2] = nxt
                    heappush(q, (nxt, v2, path))
    return distances, out_path

In [404]:
def excess(N: Network, f: Dict[Tuple[object, object], int]) -> Dict[object, int]:
    # Initialize excess to be supply
    excess = {v: N.b[v] for v in N.V}
    for (u, v), val in f.items():
        excess[u] -= val
        excess[v] += val
        
    return excess
    
def init_costs(N):
    c_f = N.u
    for (i, j) in c_f.keys():
        if (j, i) not in c_f:
            c_f[(j, i)] = 0
    return c_f

def init(N: Network):
    f = {e: 0 for e in N.E}
    p = {v: 0 for v in N.V}
    K = int(np.max(list(N.c.values())))
    return f, p, K

def reduced_cost(N: Network, u_f, p: Dict[object, int]) -> Dict[Tuple[object, object], int]:
    reduced_costs = {}
    for (u, v) in u_f.keys():
        if (u, v) in N.c:
            reduced_costs[(u, v)] = N.c[(u, v)] + p[u] - p[v]
        else:
            reduced_costs[(u, v)] = -N.c[(v, u)] - p[v] + p[u]
    return reduced_costs
    

def admissible_edges(c_f, K):
    return [e for (e, c) in c_f.items() if c >= K]

def excess_nodes(e_f, K):
    S_f = [i for (i, val) in e_f.items() if val >= K]
    T_f = [i for (i, val) in e_f.items() if val <= -K]
    return S_f, T_f

def update_potentials(p, distances):
    for i in p.keys():
        p[i] += distances[i]

def saturate_neg_cost_admissible(c_p, u_f, f, K) -> N:
    ext = {}
    for (u, v), c in u_f.items():
        if c >= K and c_p[(u,v)] < 0:
            f[(u, v)] = N.u[(u, v)]    # Saturate edge
            u_f[(u, v)] = 0            # Zero residual in forward direction
            ext[(v, u)] = N.u[(u,v)]   # Full residual in backward direction

    u_f.update(ext)
    
def augment_flow_along_path(P, f, u_f, K):
    for (i, j) in P:
        if (i, j) in f:
            f[(i, j)] += K
            u_f[(i, j)] -= K
            if (j, i) in u_f:
                u_f[(j, i)] += K
            else:
                u_f[(j, i)] = K
        else:
            f[(j, i)] -= K
            u_f[(j, i)] -= K
            if (i, j) in u_f:
                u_f[(i, j)] += K
            else:
                u_f[(i, j)] = K
    
def capacity_scaling(N):
    
    # Init zero flow and potentials
    f, p, K = init(N)
    u_f = copy(N.u)
    
    while K >= 1:
        # Compute reduced costs w.r.t potentials p
        c_p = reduced_cost(N, u_f, p)
        #print(f"Residual capacities: {u_f}\n")
        #print(f"Reduced costs: {c_p}\n")
        #print(f"Flow: {f}\n")
        # Saturate admissible edges with negative reduced cost
        saturate_neg_cost_admissible(c_p, u_f, f, K)
        
        # Compute admissible edges, and surplus/deficit nodes above scaling threshold
        e_f = excess(N, f)
        S_f, T_f = excess_nodes(e_f, K)  
        
        while len(S_f) > 0 and len(T_f) > 0:
            s = S_f[0]
            t = T_f[0]
            
            adj = {e: c for (e, c) in c_p.items() if u_f[e] >= K}
            D, P = dijkstra(s, t, adj)
            
            update_potentials(p, D)
            augment_flow_along_path(P, f, u_f, K)
            
            e_f = excess(N, f)
            S_f, T_f = excess_nodes(e_f, K)    

        K //= 2
        
    return f, p, reduced_cost(N, u_f, p)

In [405]:
f, p, c_p = capacity_scaling(N)

In [406]:
def feasibility_check(N, f):
    assert np.all(np.array(list(excess(N, f).values())) == 0)
    assert np.all(np.array(list(f.values())) >= 0)
    assert np.all(np.array(list(f.values())) <= np.array(list(N.u.values())))

In [407]:
def optimality_check(N, f, p):
    primal = np.sum([f[(i,j)]*N.c[(i,j)] for (i,j) in N.E])
    dual = -np.sum([p[i] * N.b[i] for i in N.V]) - np.sum([N.u[(i,j)] * max(0, p[j] - p[i] - N.c[(i,j)]) for (i,j) in N.E])
    assert primal == dual
    return primal

In [408]:
feasibility_check(N, f)

In [409]:
optimality_check(N, f, p)

150

In [412]:
import networkx as nx

In [415]:
G = nx.DiGraph()
G.add_node("a", demand=-5)
G.add_node("d", demand=5)
G.add_edge("a", "b", weight=3, capacity=4)
G.add_edge("a", "c", weight=6, capacity=10)
G.add_edge("b", "d", weight=1, capacity=9)
G.add_edge("c", "d", weight=2, capacity=5)
flowCost = nx.min_cost_flow_cost(G)

In [416]:
flowCost

24

In [427]:
list(G.edges)

nodes = [0, 1, 2, 3]
edges = [(0,1), (0,2), (1,3), (2,3)]
capacities = [4, 10, 9, 5]
costs = [3, 6, 1, 2]
supplies = [5, 0, 0, -5]

N = Network(nodes, edges, capacities, costs, supplies)  

In [428]:
f, p, c_p = capacity_scaling(N)

In [429]:
feasibility_check(N, f)

In [430]:
optimality_check(N, f, p)

24

In [431]:
N.u

{(0, 1): 4,
 (0, 2): 10,
 (1, 3): 9,
 (2, 3): 5,
 (0, -1): 9999,
 (1, -1): 9999,
 (2, -1): 9999,
 (3, -1): 9999,
 (-2, 0): 9999,
 (-2, 1): 9999,
 (-2, 2): 9999,
 (-2, 3): 9999,
 (-1, -2): 9999}